# Quick intro

This is a jupyter notebook for testing / coding. So far, each code block is a separate test; unlike an ordinary notebook, they are not meant to run sequentially.

First, test to see jupyter is running correctly at all:


In [ ]:
import os
import sys

print("hi!!!")
print("hi!!!")
print("hi!!!")
print("hi!!!")
print("hi!!!")

Next, running "toypyro". This started out as a copy of Fritz's code but I hope it evolves into a working version of ours.

In [31]:
from importlib import reload
import ei #import *
reload(ei)
from ei import *

trainGuide()

Yes, I will run.
(tensor([[ 68.,  20.,  25.,  56.],
        [ 24.,  52.,  82., 164.],
        [ 70., 100.,  83., 135.],
        [165.,  95.,  82., 225.],
        [263., 136.,  51., 114.],
        [105.,  84., 133., 142.],
        [105.,  44., 220., 172.],
        [127., 326., 190., 212.],
        [129., 128., 175., 225.],
        [225., 129., 167., 236.]]), tensor([[ 54.,  44.,  71.],
        [ 72., 108., 142.],
        [119., 102., 167.],
        [179., 178., 210.],
        [175., 172., 217.],
        [130., 148., 186.],
        [137., 186., 218.],
        [236., 265., 354.],
        [183., 205., 269.],
        [231., 245., 281.]]))
infoToM:16,torch.Size([16, 16])
infoToM:22,torch.Size([22, 22])
substep:16,torch.Size([16, 16]),torch.Size([22, 22])
infoToM:6,torch.Size([6, 6])
det:58.03134536743164
infoToM:22,torch.Size([22, 22])
substep:16,torch.Size([16, 16]),torch.Size([22, 22])
infoToM:6,torch.Size([6, 6])
det:58.03134536743164
infoToM:22,torch.Size([22, 22])
substep:16,torch.Size(

/anaconda3/lib/python3.7/site-packages/pyro/util.py:184: UserWarning: Found vars in model but not guide: {'sdprc'}
  warnings.warn("Found vars in model but not guide: {}".format(model_vars - guide_vars - enum_vars))
/anaconda3/lib/python3.7/site-packages/pyro/util.py:222: UserWarning: Found plate statements in guide but not model: {'precincts3', 'precinctsg2'}
  warnings.warn("Found plate statements in guide but not model: {}".format(guide_vars - model_vars))


RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation

Next, run "polytopize", which has all of my pre-Fritz attempts to code.

In [4]:
import sys
import hessian

from importlib import reload
import polytopize #import *
reload(polytopize)
from polytopize import *

import tests


Yes, I will run.
Yes, I will run.
ge fail
loc tensor([[ 0.3147],
        [ 2.4440],
        [-4.0535],
        [ 2.2007]])
polytopedLoc tensor([[ 2.1063e-01,  2.1136e-02],
        [ 2.8514e-01, -1.8626e-09],
        [ 3.4959e-01,  9.9957e-02],
        [ 3.5462e-01,  1.1892e-02],
        [ 2.5462e+00,  3.2228e-01]])
ge fail
loc tensor([[-5.8855],
        [-5.2805],
        [ 5.4654],
        [ 0.1889]])
polytopedLoc tensor([[ 2.8885e-01,  1.1871e+00],
        [ 8.5629e-01,  2.0505e+00],
        [ 3.9984e-01, -1.4901e-08],
        [ 9.5542e-01,  1.5538e+00],
        [ 1.0063e+00,  9.9522e-01]])
Reloading cmult...
callable? <bound method TorchDistributionMixin.__call__ of Multinomial()>
callable? <bound method TorchDistributionMixin.__call__ of TorchCMult()>
Sampling multinomial: tensor([1., 2.])
Sampling cm2: tensor([0., 3.])
tensor(5.6022, grad_fn=<NegBackward>) tensor([[112.2500]])


In [ ]:
loc2

result = polytopize(R,C,loc2,indep)

loc3 = depolytopize(R,C,result,indep)

print(loc2)
print(result-indep)
print(loc3)

Finally, this is a cell for directly experimenting with pytorch and/or pyro. Basically, for getting the tensor syntax right. 

In [ ]:
import torch
import cmult
import pyro
from importlib import reload
reload(cmult)
from cmult import CMult

#print(CMult(probs=torch.tensor([1., 1., 1., 1.])).log_prob(torch.tensor([1.2, 1.5, 1., 1.])))

#print(CMult(100, torch.tensor([ 1., 1., 1., 1.])).sample())

R, C = (3,4)

y = pyro.distributions.Normal(0.,4.).sample(torch.Size([R-1,C-1]))


w = torch.cat((y,-y.sum(1).unsqueeze(1)),1)
w = torch.cat((w,-w.sum(0).unsqueeze(0)),0)

#print(w)
b = w.argmax()
print(b)
print(w,w[b//C,b % C])

tt = torch.zeros(R,C)
print(tt,R,C,tt[:(R-1),:(C-1)])

## Overview

Most SVI problems in pyro are coded as a model, a generic guide (such as: multivariate Gaussian in all parameters), and specific observations/data (passed as arguments to svi.step). For EI, that's going to be different; the observations are going to be built into the guide function, leaving nothing to include in the "data" argument to svi.step.

That means there is a lot of work for the guide to do. As usual, it must establish reasonable distributional families for the posterior of each of the hyperparameters. But for the latent parameters, the job of the guide is to take a "relative strength" number for each race/candidate/precinct combo, and turn that into a number of votes for each combo, such that those numbers obey all the constraints set by observations. This means that for each precinct (considered separately), the latent guide must:

-Find the "center point" where candidate preference is independent of race.

-Find the "basis vectors" (actually, there are more than enough of them to form a basis) which determine the directions to move in the space.

-For any given set of "relative strengths" which is a distance $d$ in a direction $\theta$, find the first constraint violated when moving in that direction, and the distance $r$ between the origin and that constraint.

-Project the "relative strengths" onto the numbers of votes, by moving $r(1-e^{-d})$ in direction $\theta$.

#### a = zs(2,2,2,2)
a[0,1,1,1] = 2
print(a[1,1])
print(a[0,1])
print(torch.max(a))
print(torch.distributions.exponential.Exponential(ts([1])).sample(4))

In [ ]:
print("hi","world")
a = zs(2,2,2,2)
a[0,1,1,1] = 2 
print(torch.max(a[0,1])) 
print(pyro.distributions.Exponential(1.).sample(torch.Size([3])))
a.add_(torch.ones(2,2,2))
print(torch.exp)